In [1]:
import os
os.chdir('emo_recog')
os.listdir('.')

['.ipynb_checkpoints',
 '.vscode',
 'CNN_FER2013Plus.py',
 'Data.py',
 'DataLoader.py',
 'ExperimentManager.py',
 'FER2013PlusLoader.py',
 'FERPlus',
 'LogManager.py',
 'ModelImplementation.py',
 'PlotManager.py',
 'TestingFER2013Plus.ipynb',
 '__pycache__']

In [3]:
import ExperimentManager, FER2013PlusLoader, CNN_FER2013Plus

AttributeError: module 'numpy.random' has no attribute 'bit_generator'

In [ ]:
# Setting data directory paths
dataDirectory = "/FERPlus/data"
datasetFolderTrain = "/FERPlus/data/FER2013Train"
datasetFolderTest = "/FERPlus/data/FER2013Test"
datasetFolderValidation = "/FERPlus/data/FER2013Valid"
labelFolderTrain = "/FERPlus/data/FER2013Train/label.csv"
labelFolderTest = "/FERPlus/data/FER2013Test/label.csv"
labelFolderValidation = "/FERPlus/data/FER2013Valid/label.csv"

In [ ]:
# Initialize all the parameters and modules necessary
# image size: 64, 64
experimentManager = ExperimentManager.ExperimentManager(dataDirectory,
                                                            "FER_Experiment_GPU_CategoricalCrossentropy_2",
                                                            verbose=True)

## Data Loading

In [ ]:
grayScale = True
preProcessingProperties = [(64,64), grayScale]

In [ ]:
# Load the training and testing data
dataLoader = FER2013PlusLoader.FER2013PlusLoader(experimentManager.logManager, preProcessingProperties)
dataLoader.loadTrainData(datasetFolderTrain, labelFolderTrain)
dataLoader.loadTestData(datasetFolderTest, labelFolderTest)
dataLoader.loadValidationData(datasetFolderValidation, labelFolderValidation)

## Custom Model Testing

In [ ]:
import tensorflow as tf 
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Input, Activation, BatchNormalization, Conv2D, MaxPool2D, Dense, Dropout, Flatten, Lambda
from tensorflow.keras.optimizers import  Adamax
from tensorflow.keras.callbacks import ModelCheckpoint,  EarlyStopping , ReduceLROnPlateau
from sklearn.metrics import r2_score
import numpy, copy

In [ ]:
tf.executing_eagerly()

In [ ]:
from tensorflow.keras import backend as K
K.set_image_data_format('channels_first')

In [ ]:
import ModelImplementation

In [ ]:
# Setting hyperparameters
batch_size = 32
num_epochs = 10
num_filters = 256

In [ ]:
def shuntingInhibition(inputs):
  inhibitionDecay = 0.5
  v_c, v_c_inhibit = inputs
  output = (v_c / (inhibitionDecay + v_c_inhibit))
  return output

In [ ]:
input_shape = dataLoader.dataTest.dataX.shape[1:]
print ("Input Shape: ", input_shape)

In [ ]:
print ("Number of Outputs: ", len(dataLoader.dataTest.labelDictionary))

In [ ]:
inputLayer = Input(shape=input_shape)

In [ ]:
num_of_outputs = len(dataLoader.dataTest.labelDictionary)

In [ ]:
# Convolutional Layers 1 & 2
conv1 = Conv2D(int(num_filters / 4), kernel_size=(3, 3), padding="same", data_format="channels_first")(inputLayer)
bn1 = BatchNormalization(axis = 1)(conv1)
actv1 = Activation("relu")(bn1)
conv2 = Conv2D(int(num_filters / 4), (3, 3), padding="same", data_format="channels_first")(actv1)
bn2 = BatchNormalization(axis = 1)(conv2)
actv2 = Activation("relu")(bn2)
mp1 = MaxPool2D(pool_size=(2, 2))(actv2)
drop1 = Dropout(0.25)(mp1)

# Convolutional layers 3 & 4
conv3 = Conv2D(int(num_filters / 2), (3, 3), padding="same", data_format="channels_first")(drop1)
bn3 = BatchNormalization(axis = 1)(conv3)
actv3 = Activation("relu")(bn3)
conv4 = Conv2D(int(num_filters / 2), (3, 3), padding="same", data_format="channels_first")(actv3)
bn4 = BatchNormalization(axis = 1)(conv4)
actv4 = Activation("relu")(bn4)
mp2 = MaxPool2D(pool_size=(2, 2))(actv4)
drop2 = Dropout(0.25)(mp2)

# Convolutional layers 5, 6, 7
conv5 = Conv2D(int(num_filters / 2), (3, 3), padding="same", data_format="channels_first")(drop2)
bn5 = BatchNormalization(axis = 1)(conv5)
actv5 = Activation("relu")(bn5)
conv6 = Conv2D(int(num_filters / 2), (3, 3), padding="same", data_format="channels_first")(actv5)
bn6 = BatchNormalization(axis = 1)(conv6)
actv6 = Activation("relu")(bn6)
conv7 = Conv2D(int(num_filters / 2), (3, 3), padding="same", data_format="channels_first")(actv6)
bn7 = BatchNormalization(axis = 1)(conv7)
actv7 = Activation("relu")(bn7)
mp3 = MaxPool2D(pool_size=(2, 2))(actv7)
drop3 = Dropout(0.25)(mp3)

# Convolutional Layers 8, 9 , 10
conv8 = Conv2D(num_filters, (3, 3), padding="same", data_format="channels_first")(drop3)
bn8 = BatchNormalization(axis = 1)(conv8)
actv8 = Activation("relu")(bn8)
conv9 = Conv2D(num_filters, (3, 3), padding="same", data_format="channels_first")(actv8)
bn9 = BatchNormalization(axis = 1)(conv9)
actv9 = Activation("relu")(bn9)
conv10 = Conv2D(num_filters , (3, 3), padding="same", activation="relu")(actv9)
conv10_inhibition = Conv2D(num_filters, (3, 3), padding="same", activation="relu")(actv9)
v_conv_inhibitted = Lambda(function=shuntingInhibition)([conv10, conv10_inhibition])
mp4 = MaxPool2D(pool_size=(2, 2))(v_conv_inhibitted)
drop4 = Dropout(0.25)(mp4)
flatten = Flatten()(drop4)
dense = Dense(200, activation="relu")(flatten)
drop5 = Dropout(0.25)(dense)
output = Dense(num_of_outputs, activation="softmax")(drop5)

model = Model(inputs=inputLayer, outputs=output)
model.summary()

In [ ]:
def fmeasure( y_true, y_pred):
  """Computes the f-measure, the harmonic mean of precision and recall.
  Here it is only computed as a batch-wise average, not globally.
  """
  return fbeta_score(y_true, y_pred, beta=1)

In [ ]:
def fbeta_score( y_true, y_pred, beta=0.5):
  from keras import backend as Kend
  """Computes the F score.
  The F score is the weighted harmonic mean of precision and recall.
  Here it is only computed as a batch-wise average, not globally.
  This is useful for multi-label classification, where input samples can be
  classified as sets of labels. By only using accuracy (precision) a model
  would achieve a perfect score by simply assigning every class to every
  input. In order to avoid this, a metric should penalize incorrect class
  assignments as well (recall). The F-beta score (ranged from 0.0 to 1.0)
  computes this, as a weighted mean of the proportion of correct class
  assignments vs. the proportion of incorrect class assignments.
  With beta = 1, this is equivalent to a F-measure. With beta < 1, assigning
  correct classes becomes more important, and with beta > 1 the metric is
  instead weighted towards penalizing incorrect class assignments.
  """
  if beta < 0:
      raise ValueError('The lowest choosable beta is zero (only precision).')

  # If there are no true positives, fix the F score at 0 like sklearn.
  if Kend.sum(Kend.round(Kend.clip(y_true, 0, 1))) == 0:
      return 0

  p = precision(y_true, y_pred)
  r = recall(y_true, y_pred)
  bb = beta ** 2
  fbeta_score = (1 + bb) * (p * r) / (bb * p + r + Kend.epsilon())
  return fbeta_score

In [ ]:
def precision( y_true, y_pred):
  from keras import backend as Kend
  """Precision metric.
  Only computes a batch-wise average of precision.
  Computes the precision, a metric for multi-label classification of
  how many selected items are relevant.
  """
  true_positives = Kend.sum(Kend.round(Kend.clip(y_true * y_pred, 0, 1)))
  predicted_positives = Kend.sum(Kend.round(Kend.clip(y_pred, 0, 1)))
  precision = true_positives / (predicted_positives + Kend.epsilon())
  return precision
    
def recall( y_true, y_pred):
  from keras import backend as Kend
  """Recall metric.
  Only computes a batch-wise average of recall.
  Computes the recall, a metric for multi-label classification of
  how many relevant items are selected.
  """
  true_positives = Kend.sum(Kend.round(Kend.clip(y_true * y_pred, 0, 1)))
  possible_positives = Kend.sum(Kend.round(Kend.clip(y_true, 0, 1)))
  recall = true_positives / (possible_positives + Kend.epsilon())
  return recall

In [ ]:
optimizer = Adamax(lr=0.002, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
model.compile(loss= "categorical_crossentropy",
              optimizer=optimizer,
              metrics=['accuracy','categorical_accuracy'])

In [ ]:
filepath=experimentManager.modelDirectory + "/weights.best.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_categorical_accuracy', verbose=1, save_best_only=True, mode='max')
early_stopping = EarlyStopping(monitor='val_loss', mode="min", patience=25)
reduce_lr = ReduceLROnPlateau(factor = 0.5, monitor='val_loss', min_lr = 1e-5, patience = 2)

callbacks_list = [checkpoint, early_stopping, reduce_lr]

In [ ]:
model.fit(dataLoader.dataTrain, dataLoader.dataValidation, epochs=10)

## CNN Building and Training

In [ ]:
cnnModel = CNN_FER2013Plus.CNN_FER2013(experimentManager, "CNN", experimentManager.plotManager)

In [ ]:
cnnModel.buildModel(dataLoader.dataTest.dataX.shape[1:], len(dataLoader.dataTest.labelDictionary))

In [ ]:
cnnModel.train(dataLoader.dataTrain, dataLoader.dataValidation, False)

In [ ]:
cnnModel.save(experimentManager.modelDirectory)

## Converting Python 2 code to Python 3
issue: can't load libraries after conversion for some reason

In [ ]:
!pip3 install 2to3

In [ ]:
# Need to convert from Python 2 to Python 3
!2to3 KEF/Controllers/PlotManager.py
# !2to3 -w KEF/Controllers/LogManager.py
#!2to3 -w KEF/Controllers/ExperimentManager.py
#!2to3 KEF/DataLoaders/FER2013PlusLoader.py
# !2to3 -w KEF/Implementations/Vision_CNN_FER2013.py
#!2to3 -w KEF/Models/Data.py
# !2to3 -w KEF/CustomObjects/losses.py
# !2to3 -w KEF/CustomObjects/metrics.py
# !2to3 -w KEF/Metrics/losses.py
#!2to3 -w KEF/Metrics/metrics.py


In [ ]:
from KEF.Controllers import ExperimentManager

In [ ]:
from KEF.DataLoaders import FER2013PlusLoader

In [ ]:
from KEF.Implementations import Vision_CNN_FER2013

In [ ]:
from google.colab import drive
drive.mount('/content/drive')